In [1]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# SCOPES: Gmail read-only
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

def authenticate_gmail():
    creds = None
    flow = InstalledAppFlow.from_client_secrets_file('C:/airflow-docker/include/secrets/gmail/credentials.json', SCOPES)
    creds = flow.run_local_server(port=0)
    service = build('gmail', 'v1', credentials=creds)
    return service

In [2]:
service = authenticate_gmail()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=605782370280-mohr8q21oavsvm9l8712s186e8o23vdf.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A50222%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=GRkyS3C99oge5V5SOYOhYMyYG1ZUpx&access_type=offline


In [3]:
def get_message_metadata(service, user_id='me', max_results=5):
    results = service.users().messages().list(userId=user_id, maxResults=max_results, q="after:2025/08/08").execute()
    messages = results.get('messages', [])

    for _, msg in enumerate(messages):
        msg_data = service.users().messages().get(userId=user_id, id=msg['id'], format='metadata', metadataHeaders=['Subject', 'From', 'Date']).execute()
        headers = msg_data.get('payload', {}).get('headers', [])

        email_info = {header['name']: header['value'] for header in headers if header['name'] in ['Subject', 'From', 'Date']}
        print(f"From: {email_info.get('From')}")
        print(f"Subject: {email_info.get('Subject')}")
        print(f"Date: {email_info.get('Date')}")

In [169]:
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
        
token_path = "C:/airflow-docker/include/secrets/gmail/token.json"

creds = Credentials.from_authorized_user_file(token_path, SCOPES)
service = build('gmail', 'v1', credentials=creds)

results = service.users().messages().list(userId='me', maxResults=5, q="after:2025/08/01 before:2025/08/04").execute()
messages = results.get('messages', [])

In [11]:
results.get('messages', [])

[{'id': '19872413177971ec', 'threadId': '19872413177971ec'},
 {'id': '1987190da5ec8871', 'threadId': '1987190da5ec8871'},
 {'id': '198717c33892b97d', 'threadId': '198717c33892b97d'},
 {'id': '198715ecc39f04e2', 'threadId': '198715ecc39f04e2'},
 {'id': '198703627fc7270f', 'threadId': '198703627fc7270f'}]

In [12]:
def get_full_messages(service, message_ids):
    messages = []
    for msg in message_ids:
        message = service.users().messages().get(userId="me", id=msg['id']).execute()
        messages.append(message)
    return messages
x = get_full_messages(service, messages)

In [ ]:
import time, threading
from datetime import  datetime, timedelta

def last_7_days_generator(start_date: datetime.date):
    for i in range(1, 8):
        yield start_date - timedelta(days=i)

def get_ids(i, gen_1, gen_2):
    start_time = time.time()
    creds = Credentials.from_authorized_user_file(token_path, SCOPES)
    service = build('gmail', 'v1', credentials=creds)
    results = service.users().messages().list(userId='me', q=f"after:{next(gen_2)} before:{next(gen_1)}").execute()
    #time.sleep(0.3)
    messages = results.get('messages', [])
    print(len(messages))
    print(f"[Thread-{i}]. Time taken: {time.time() - start_time:.4f} sec") 
   

In [ ]:
fetch_threads = []
shutdown_event = threading.Event()
x = datetime.now().date()  # just date
y = x - timedelta(days=1)
gen_1 =  last_7_days_generator(x)
gen_2 =  last_7_days_generator(y)
for i in range(7):
    t = threading.Thread(target=get_ids, args=(i+1, gen_1, gen_2))
    t.start()
    fetch_threads.append(t)
    
# Signal writer to shut down
shutdown_event.set()

30
[Thread-3]. Time taken: 2.9985 sec
23
[Thread-2]. Time taken: 3.0616 sec
35
[Thread-1]. Time taken: 3.0727 sec
28
[Thread-4]. Time taken: 3.0741 sec
23
[Thread-5]. Time taken: 3.0904 sec
38
[Thread-6]. Time taken: 3.1190 sec
34
[Thread-7]. Time taken: 2.9161 sec
